In [91]:
%%capture
#### Load Libraries (without cell output)

import autograd.numpy as np # import autograd wrapped numpy
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import sklearn
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import neighbors, metrics
from sklearn.preprocessing import scale

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm

from sklearn.impute import SimpleImputer

In [92]:
def RMSE(x, y):
    MSE = ((y - x) ** 2).mean()
    return np.sqrt(MSE)

claims = pd.read_csv("./Qualification_Package/Claims_Years_1_to_3.csv")

In [93]:
def preprocess_xy(dataframe):
    claims = dataframe.copy(deep=True)
    claims['pol_pay_freq'] = claims['pol_pay_freq'].replace( {'Biannual': 2, 'Yearly': 1, 'Monthly': 12, 'Quarterly': 4} )
    claims['pol_payd'] = claims['pol_payd'].replace( {'No': 0, 'Yes': 1} )
    claims['drv_sex1'] = claims['drv_sex1'].replace( {'M': 1, 'F': 0} )
    claims['vh_type'] = claims['vh_type'].replace( {'Tourism': 1, 'Commercial': 0} )
    claims['drv_drv2'] = claims['drv_drv2'].replace( {'No': 0, 'Yes': 1} )

    objects = claims.select_dtypes(['object'])
    categorical = claims.select_dtypes(['int64'])
    continuous = claims.select_dtypes(['float64'])

    objects_filled = SimpleImputer(missing_values=np.nan, strategy='most_frequent').fit_transform(objects)
    categorial_filled = SimpleImputer(missing_values=np.nan, strategy='median').fit_transform(categorical)
    continuous_filled = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(continuous)

    objects_filled = pd.DataFrame(objects_filled, columns=objects.columns.to_list())
    categorial_filled = pd.DataFrame(categorial_filled, columns=categorical.columns.to_list())
    continuous_filled = pd.DataFrame(continuous_filled, columns=continuous.columns.to_list())

    objects_filled.drop(columns=['vh_make_model'])

    design_matrix = pd.get_dummies(objects_filled, columns=['pol_usage', 'drv_sex2', 'vh_fuel'], dtype=int) #'vh_make_model', 
    objects_design = design_matrix.drop(columns=['id_policy'])

    df = pd.concat([objects_design, categorial_filled, continuous_filled], axis = 1)

    x = df.iloc[:,:-1]
    y = df.iloc[:,-1]

    x_train, x_test, y_train, y_test = train_test_split(
        x,
        y, 
        train_size = 0.8,
        test_size = 0.2, # train is 75%, test is 25% 
        random_state = 0, # stratify = y,
    )
    return x_train, x_test, y_train, y_test

In [94]:
list_vh_make_model = ['vh_make_model_aaykjdjgdzrrdvxz',
 'vh_make_model_abacekzzrkhtgpcp',
 'vh_make_model_abcepdrvvynjsufa',
 'vh_make_model_aceqpjprqgzhffuw',
 'vh_make_model_adzzjitkyqlberpu',
 'vh_make_model_aewtdnpoiopumymt',
 'vh_make_model_agowcnternxraavr',
 'vh_make_model_aifsqdniwqmcuqpv',
 'vh_make_model_ajtardhciglimsdi',
 'vh_make_model_alrfnehgsdtsunhm',
 'vh_make_model_ambmbeydwsdljdcc',
 'vh_make_model_ammbrasbxojlitmt',
 'vh_make_model_anqyvxqouldudiww',
 'vh_make_model_aparvvfowrjncdhp',
 'vh_make_model_arcsdpohuzvikyaw',
 'vh_make_model_arfzuuojdtlgxehv',
 'vh_make_model_asmpttrlkodaejic',
 'vh_make_model_aysnuezuqgjioyyf',
 'vh_make_model_aywlgifrijfokyzu',
 'vh_make_model_biqzvbfzjivqmrro',
 'vh_make_model_blcuqlgntjavsyhs',
 'vh_make_model_bnvgzfegimthyhyo',
 'vh_make_model_bsiyfrkwdyptmwji',
 'vh_make_model_bvuzvpriwqlnbjxt',
 'vh_make_model_byvoguptigfevpyy',
 'vh_make_model_cazrxylvhylncoze',
 'vh_make_model_cbmmnlpqoyyursux',
 'vh_make_model_celpzeaubkxaxxbx',
 'vh_make_model_cgkclpnidlmetsrb',
 'vh_make_model_cgrdxjyaxssrszjz',
 'vh_make_model_cjcthmigqkejxuzi',
 'vh_make_model_clmcokjtplrbzvuh',
 'vh_make_model_cmmuslxsfluvfyof',
 'vh_make_model_cnicorpxweynumqk',
 'vh_make_model_cnlvybtdupkcwczn',
 'vh_make_model_coufviypetbrtevy',
 'vh_make_model_cqdmtwkacajclcml',
 'vh_make_model_cqewccykrcmvawlo',
 'vh_make_model_csxjshhnfbtgjcgm',
 'vh_make_model_ctachoeiozcpkmst',
 'vh_make_model_cuxaapvakeemmbaa',
 'vh_make_model_cwshqcgmaazzefkx',
 'vh_make_model_cxxzogxxkmkjwqui',
 'vh_make_model_dbtkrhmbfxpkqbau',
 'vh_make_model_dcjzdpoxqvgnjpmi',
 'vh_make_model_demgvtbzilochupd',
 'vh_make_model_dgwbxitzfzbegnoc',
 'vh_make_model_dgwtezteqyzzylho',
 'vh_make_model_dhjmmmtnpcnalzna',
 'vh_make_model_dhxftxnxtxlgqcqb',
 'vh_make_model_disoykeofihapsal',
 'vh_make_model_djyptluftbfkxtjd',
 'vh_make_model_dlbnpwopifytzerl',
 'vh_make_model_dllcylnkzeegtsgr',
 'vh_make_model_dlrodwgixwmoquny',
 'vh_make_model_dlrvgwmumnwcjixm',
 'vh_make_model_dnvcqpxxzahdhbvy',
 'vh_make_model_doohwubeqhbkevhr',
 'vh_make_model_dpklliwcxycpfriu',
 'vh_make_model_drptidaltxzxopwv',
 'vh_make_model_dsqmtbudvjtnnjwq',
 'vh_make_model_dtpbahjtnmyuxqno',
 'vh_make_model_dvshwarqhxfcgwfd',
 'vh_make_model_dweqmfoluivgiayj',
 'vh_make_model_dwhlbcevejvegsob',
 'vh_make_model_dyzvyrmcdyybbddd',
 'vh_make_model_ebdcmhmtqnfkaalo',
 'vh_make_model_efiskxgaocgqqjvr',
 'vh_make_model_ehapkksqqcbofeid',
 'vh_make_model_ehtbxdjhvcwdapsg',
 'vh_make_model_eivjhovgfnfctgjy',
 'vh_make_model_ejbxcyhffvcouoxd',
 'vh_make_model_ejeggxbwhufjtjhd',
 'vh_make_model_enmicxqiumbpozpk',
 'vh_make_model_eokuiduvnrtzavmr',
 'vh_make_model_esilvarzflhfmjhh',
 'vh_make_model_eslneidrjqwzpqhd',
 'vh_make_model_ettwalwfkzvwdasa',
 'vh_make_model_eudwptcohxaazhpt',
 'vh_make_model_ewkcexkqpsyfnugi',
 'vh_make_model_exkqtrkthhgvjqdl',
 'vh_make_model_eyaqhofitsegmcwi',
 'vh_make_model_eyrwkwxecpzxzscp',
 'vh_make_model_fadjogsnmecatcfb',
 'vh_make_model_fdbwfjqkichwdebq',
 'vh_make_model_fklewvbxuecmupxn',
 'vh_make_model_flpmjcetsinyjimc',
 'vh_make_model_fpfzaadmykntrupr',
 'vh_make_model_fuddhlszptfmosir',
 'vh_make_model_fuwhdjmdexrstmmo',
 'vh_make_model_gapclpflkdsbeorm',
 'vh_make_model_gdaxhrlhuilhiijt',
 'vh_make_model_gfhjqtkgvomiygvx',
 'vh_make_model_ggaokfjtqxyctvok',
 'vh_make_model_gguphuccgeqyojbl',
 'vh_make_model_giyhzprslgbwsaeu',
 'vh_make_model_gjbalugsikirqoam',
 'vh_make_model_gjblfwqtnckjletn',
 'vh_make_model_gjchrdhbeixppooh',
 'vh_make_model_gjpgirzuabhfpkjd',
 'vh_make_model_gjxmrfgnorpfspbb',
 'vh_make_model_gkxcvooedomgcagl',
 'vh_make_model_gmzbnaysqjpkzqbt',
 'vh_make_model_gpclrtlzecazeeev',
 'vh_make_model_gqfadgvnztixxbmv',
 'vh_make_model_gsooyxmnwsucrksh',
 'vh_make_model_guiimarisyyjqnfg',
 'vh_make_model_gujwvdfcmmqcwxfi',
 'vh_make_model_gwptulznqgygeegy',
 'vh_make_model_gxgjyxrnnugizdvf',
 'vh_make_model_gzebcnjcmqioqcjb',
 'vh_make_model_hayciibjzwapccnb',
 'vh_make_model_hcfpedolsygjsofb',
 'vh_make_model_hcoxxbfccserxklx',
 'vh_make_model_hhrmdevbfqiebnum',
 'vh_make_model_hkazsxqvbtmawovu',
 'vh_make_model_hpohizpkyzvwunni',
 'vh_make_model_hselphnqlvecmmyx',
 'vh_make_model_htedybhazfjiueyj',
 'vh_make_model_htppstzpipwjtuia',
 'vh_make_model_huoicgalccftwyvz',
 'vh_make_model_hwldevoubgzgbhgs',
 'vh_make_model_hywzsmogbhnfcaxk',
 'vh_make_model_iemmvtjtejhlteqa',
 'vh_make_model_iknapxqudqotqiig',
 'vh_make_model_infvsqmvfzjpyfae',
 'vh_make_model_innngarflvbnwntw',
 'vh_make_model_ioqpncqqlflrjzkj',
 'vh_make_model_iqepotyqjqeebzix',
 'vh_make_model_ismjlsoibleinjdp',
 'vh_make_model_iulvirmzdntweaee',
 'vh_make_model_iwhqpdfuhrsxyqxe',
 'vh_make_model_ixbrfaoerogqomah',
 'vh_make_model_ixwsqebjjdlxcqsq',
 'vh_make_model_jakvzvdollijyhwm',
 'vh_make_model_jcefoutonncubdss',
 'vh_make_model_jeckddxjsdolnuhe',
 'vh_make_model_jepialiqqsttgcid',
 'vh_make_model_jgkpiuuctpywtrlh',
 'vh_make_model_jhdjdpthkztnjvmb',
 'vh_make_model_jiyzqszfywhdfsil',
 'vh_make_model_jjjvjaxpzvlbryfd',
 'vh_make_model_jjycmklnkdivnypu',
 'vh_make_model_jkarjtlhihuxqzfm',
 'vh_make_model_jkhjcfudwqurdoex',
 'vh_make_model_jlxizhsfukrheysf',
 'vh_make_model_jmycebfjwrkqwsxi',
 'vh_make_model_johsjccpkithubii',
 'vh_make_model_joosvbazdbslkqgx',
 'vh_make_model_jrwdpzrmxqlzzepk',
 'vh_make_model_jrwemlawxsvnwrxv',
 'vh_make_model_jsudrcgsrfddwixw',
 'vh_make_model_kalfshwbcuoobdwe',
 'vh_make_model_kbgblyclstrmicux',
 'vh_make_model_kdsbtuikoaulynsu',
 'vh_make_model_kfurwythfncqbrxs',
 'vh_make_model_kguahfjnmerrbtpp',
 'vh_make_model_kjdumkaiaeblbxtt',
 'vh_make_model_kjhuznifzeghfdra',
 'vh_make_model_kmlnlefquqpparsa',
 'vh_make_model_kpciudedjlrqsfte',
 'vh_make_model_kqxycgbergacgcei',
 'vh_make_model_kvcddisqpkysmvvo',
 'vh_make_model_kwxjejihbgmtnagf',
 'vh_make_model_kzwthrslljkmbqur',
 'vh_make_model_lcokgbxbqigkqzcw',
 'vh_make_model_ldejndeewhhlcvgc',
 'vh_make_model_lfzbrhthlxhnmhva',
 'vh_make_model_llkwlxfjdmrqmdgq',
 'vh_make_model_lpwtmtiwkgbwhufg',
 'vh_make_model_lqkdgbosdzrtitgx',
 'vh_make_model_lqqciehjjdtelpwa',
 'vh_make_model_luwiodhzrjjobjlw',
 'vh_make_model_lwrjcljtxkokvnes',
 'vh_make_model_mbytpqiuixyvpaab',
 'vh_make_model_mcadxmmocjhzzbtt',
 'vh_make_model_mdxtphkujabwpjeu',
 'vh_make_model_meratbpknllwoefn',
 'vh_make_model_mizxbkgdiuoehddq',
 'vh_make_model_mjwrreshlbmzkwmc',
 'vh_make_model_moayoogjmiizcbez',
 'vh_make_model_mshhupropfijhilz',
 'vh_make_model_nbxjozrynlospbso',
 'vh_make_model_ngombkqqomblyxwv',
 'vh_make_model_nhoebceeiacnmvym',
 'vh_make_model_nhwgapjtnadqqaul',
 'vh_make_model_nilvygybpajtnxnr',
 'vh_make_model_nkueyjctyasmotny',
 'vh_make_model_nmhahirmbvqxhxgg',
 'vh_make_model_nmkzmncfytfwyfvt',
 'vh_make_model_nofmcfnaiuzlqgrk',
 'vh_make_model_nolayrxwnjwzgtoo',
 'vh_make_model_nrmzpcqkbzgmsdeo',
 'vh_make_model_nruhduwvuytxnfvh',
 'vh_make_model_nsymgnybdjqxudvj',
 'vh_make_model_nwaavqeweeqaryzv',
 'vh_make_model_nxwedpnhirijkodc',
 'vh_make_model_nyrtstlobluggnkw',
 'vh_make_model_nzxlhibmhrtafeav',
 'vh_make_model_obvxygchobqafuzw',
 'vh_make_model_odpuaztxnyumdvvc',
 'vh_make_model_ofrkezlcbbluncri',
 'vh_make_model_ohxrgpugowiyinhv',
 'vh_make_model_olupoctwepebdqqo',
 'vh_make_model_onzjhhtppsfaiacz',
 'vh_make_model_oryfrzxilushvigq',
 'vh_make_model_otrziwxmbpndmyaa',
 'vh_make_model_owxrlgxbigikfgtm',
 'vh_make_model_ozmdlzfsareqmkon',
 'vh_make_model_ozpyjjijxdpztngv',
 'vh_make_model_pbrroilhklrifbwq',
 'vh_make_model_pdljbgzzhxrhnqmu',
 'vh_make_model_pjbnwqhnqczouirt',
 'vh_make_model_ponwkmeaxagundzq',
 'vh_make_model_prtnwsypyfnshpqx',
 'vh_make_model_pselomoxubpkknqo',
 'vh_make_model_ptaxsjwbissrpvdm',
 'vh_make_model_pustczakchcimwuy',
 'vh_make_model_pyholyswkkqjmxlj',
 'vh_make_model_pyykjiriqrhjduly',
 'vh_make_model_qbztetcodwhfmoyg',
 'vh_make_model_qewzxgvvhqhkfcxe',
 'vh_make_model_qfvolfbvalczrcko',
 'vh_make_model_qgnqfinpenszbzig',
 'vh_make_model_qnesuhpxsptzihzg',
 'vh_make_model_qnixeczkijjyiprb',
 'vh_make_model_qpcebxmotqhildhx',
 'vh_make_model_qukbrubjquwstnyf',
 'vh_make_model_quslbttvcitxzeiy',
 'vh_make_model_qvwenzdmnwecdiql',
 'vh_make_model_qxtqrwxfvuenelml',
 'vh_make_model_qzgaezfhutbcnkuf',
 'vh_make_model_rabwrzdzwjjdhbmx',
 'vh_make_model_rcxmbwwsxkkkyyjs',
 'vh_make_model_rgrpzewhrznrqrna',
 'vh_make_model_rguedwefqmzdxowu',
 'vh_make_model_rnrkbyojyiepdvqv',
 'vh_make_model_rqaprgqcktgrlxnv',
 'vh_make_model_rqklbykswxeuovdn',
 'vh_make_model_rrlvhbnzrdtphqnl',
 'vh_make_model_rrqbtdjvuwwxtusj',
 'vh_make_model_rthsjeyjgdlmkygk',
 'vh_make_model_rulqevsymrlwrsrz',
 'vh_make_model_ruposftqgswlcyou',
 'vh_make_model_rwrevaiebpmviwqz',
 'vh_make_model_rwtwnvhjqabvovnz',
 'vh_make_model_rxyndewyvbophaku',
 'vh_make_model_saempmkfulqhwfqk',
 'vh_make_model_sdottmimvqvfhzlk',
 'vh_make_model_sguprofjftozaujc',
 'vh_make_model_sioekxjbocpzrjzi',
 'vh_make_model_skwelgffvlzgmbro',
 'vh_make_model_smynsodmtrrubpqq',
 'vh_make_model_snpaaoiipfuxmvol',
 'vh_make_model_snsnxmucuccvqfvz',
 'vh_make_model_spbjeokdemicpdey',
 'vh_make_model_squxtuwvjnzbhzsc',
 'vh_make_model_srgqbkjrwdbikmzq',
 'vh_make_model_sutdaojcvfqmjnwg',
 'vh_make_model_svmjzfcsvgxiwwjt',
 'vh_make_model_swjkmyqytzxjwgag',
 'vh_make_model_synvsxhrexuyxpre',
 'vh_make_model_szlkmablxrjoubla',
 'vh_make_model_tceovgpqjjopitor',
 'vh_make_model_tddtoayhfpdtxokp',
 'vh_make_model_tdgkjlphosocwbgu',
 'vh_make_model_tdozuksvtvtqcykp',
 'vh_make_model_tkqxtjbbrzagooya',
 'vh_make_model_tlhipnhcbdhvhgyw',
 'vh_make_model_toqaaqswchaiyhsk',
 'vh_make_model_tupmlwnkgjcgcmuv',
 'vh_make_model_uahuglbjdtacoqjt',
 'vh_make_model_ubttjiaeeuwzcclq',
 'vh_make_model_udlfdefgndowttah',
 'vh_make_model_ueujqvpwszzhovbj',
 'vh_make_model_ufpwnqycocwwbgqi',
 'vh_make_model_ugwrafmvdavbsrzl',
 'vh_make_model_uhkhghxuorryhlis',
 'vh_make_model_uhqwwluaswbuqqjc',
 'vh_make_model_ujgldnoigollndkj',
 'vh_make_model_ukqbsscpgbfatdhs',
 'vh_make_model_unmxysjyilftwsvy',
 'vh_make_model_uotpflqyvprslxjc',
 'vh_make_model_urullqqbaabxllxl',
 'vh_make_model_usjukvawgoqplrph',
 'vh_make_model_usmpkujeknoxdqrc',
 'vh_make_model_uureltetaotxxdji',
 'vh_make_model_uwwrbkmjbjyxutfq',
 'vh_make_model_uzmldekmvczimsrj',
 'vh_make_model_uzqtsirvtxcfqnbp',
 'vh_make_model_uzrhbfduaqijosql',
 'vh_make_model_vdetuihriafhetdl',
 'vh_make_model_vdwfxzfzbybhsmay',
 'vh_make_model_vikmjrynreazqubj',
 'vh_make_model_vjarzxzevsdnftcl',
 'vh_make_model_vjxfnjqgvugwjhia',
 'vh_make_model_vnxfpyxuciadydrl',
 'vh_make_model_vokpwtikxckeemdi',
 'vh_make_model_vudjowytbogxkrcy',
 'vh_make_model_vylopbnfewdzeury',
 'vh_make_model_wchxrbrhstsmhdsk',
 'vh_make_model_wehkqzwvbeonajcu',
 'vh_make_model_wgruytvmfzalzrtb',
 'vh_make_model_whcqrtwarljaqocm',
 'vh_make_model_wiyyghhcxezudyxg',
 'vh_make_model_wkzwidzltxinpgen',
 'vh_make_model_wpsyqubfrhdspxkx',
 'vh_make_model_wrvfnbtdqgpsnzic',
 'vh_make_model_wsfgxnwhxftjhpxw',
 'vh_make_model_wxzfbqtarfurwcfw',
 'vh_make_model_wyopladghryqlrlb',
 'vh_make_model_wyoxchtoecahbyjm',
 'vh_make_model_xaklvfxsplowrglp',
 'vh_make_model_xdeuhuabvdhjipnp',
 'vh_make_model_xgvsuftfggoojbdp',
 'vh_make_model_xhczitnzxmxxebeq',
 'vh_make_model_xiesnbkcyzrpzlyq',
 'vh_make_model_xjaddkudsebowzen',
 'vh_make_model_xjnjmxyqemqqiejp',
 'vh_make_model_xjsozzwcppavldee',
 'vh_make_model_xkqlmdeookbxzzhv',
 'vh_make_model_xkzehzohmfrsmolg',
 'vh_make_model_xoetbcowyxwukawr',
 'vh_make_model_xpxsjmglcvcsxwdy',
 'vh_make_model_xsmqbeukcqahbfgl',
 'vh_make_model_xyuwuxlpirkzkqdb',
 'vh_make_model_xzdsapxqliboezbc',
 'vh_make_model_ybmkbrazyartpatx',
 'vh_make_model_yfalryaixpzfoihd',
 'vh_make_model_ygqjgwzgeierkcpj',
 'vh_make_model_ygsnfanduarpqvrn',
 'vh_make_model_yhcelpjnbxpsmoez',
 'vh_make_model_ypgcfauffeqpeerz',
 'vh_make_model_yqrgxthbbzmruvwy',
 'vh_make_model_yttvzqeuddvehiqu',
 'vh_make_model_yvlkrzgjhwrlyihc',
 'vh_make_model_yybjogamsfqljfpu',
 'vh_make_model_yzdbjmwwtofxmpaz',
 'vh_make_model_zagpnfpbwgeyeufr',
 'vh_make_model_zakviitdfvxsgkow',
 'vh_make_model_zbprczwzmlxgqykc',
 'vh_make_model_zcqptmhakcmihiry',
 'vh_make_model_zedpyrfkmhzqxmaz',
 'vh_make_model_zfylmujpvzgqqfxo',
 'vh_make_model_zkreetxvsoihwkgo',
 'vh_make_model_zoypfizhpbtpjwpv',
 'vh_make_model_zqruwnlzuefcpqjm',
 'vh_make_model_zqswdfwtkyehitft',
 'vh_make_model_zspzyfdefowgwddf',
 'vh_make_model_ztwsplndgicacmuu',
 'vh_make_model_zxvcbwcwoqnkxxbs',
 'vh_make_model_zydfvjqmmwhyfuyy',
 'vh_make_model_zzlzzujtugbfpsvv',
 'pol_usage_AllTrips',
 'pol_usage_Professional',
 'pol_usage_Retired',
 'pol_usage_WorkPrivate',
 'drv_sex2_0',
 'drv_sex2_F',
 'drv_sex2_M',
 'vh_fuel_Diesel',
 'vh_fuel_Gasoline',
 'vh_fuel_Hybrid',
 'Unnamed: 0',
 'year',
 'pol_duration',
 'pol_pay_freq',
 'pol_payd',
 'drv_sex1',
 'drv_age1',
 'drv_age_lic1',
 'drv_drv2',
 'vh_age',
 'vh_type',
 'population',
 'pol_no_claims_discount',
 'drv_age2',
 'drv_age_lic2',
 'vh_speed',
 'vh_value',
 'vh_weight',
 'town_surface_area']

In [95]:
all_cols = ['vh_make_model_aawqanlavsjfqrne',
 'vh_make_model_abacekzzrkhtgpcp',
 'vh_make_model_abcepdrvvynjsufa',
 'vh_make_model_aceqpjprqgzhffuw',
 'vh_make_model_adhoqfsfdpetomvs',
 'vh_make_model_adzzjitkyqlberpu',
 'vh_make_model_aewtdnpoiopumymt',
 'vh_make_model_aifsqdniwqmcuqpv',
 'vh_make_model_aivacsqryguqpdib',
 'vh_make_model_ajktbllxjzfdtwpy',
 'vh_make_model_ajtardhciglimsdi',
 'vh_make_model_alrfnehgsdtsunhm',
 'vh_make_model_ambmbeydwsdljdcc',
 'vh_make_model_ammbrasbxojlitmt',
 'vh_make_model_anqyvxqouldudiww',
 'vh_make_model_aparvvfowrjncdhp',
 'vh_make_model_arcsdpohuzvikyaw',
 'vh_make_model_arfzuuojdtlgxehv',
 'vh_make_model_asmpttrlkodaejic',
 'vh_make_model_atsglyxkfbaztzlj',
 'vh_make_model_ayeiibefzqqbyksg',
 'vh_make_model_aysnuezuqgjioyyf',
 'vh_make_model_aywlgifrijfokyzu',
 'vh_make_model_bikffjqejohkyhat',
 'vh_make_model_biqzvbfzjivqmrro',
 'vh_make_model_blcuqlgntjavsyhs',
 'vh_make_model_bnvgzfegimthyhyo',
 'vh_make_model_bsiyfrkwdyptmwji',
 'vh_make_model_bvuzvpriwqlnbjxt',
 'vh_make_model_bwpieeluivljdtai',
 'vh_make_model_byvoguptigfevpyy',
 'vh_make_model_cazrxylvhylncoze',
 'vh_make_model_cbmmnlpqoyyursux',
 'vh_make_model_celpzeaubkxaxxbx',
 'vh_make_model_cfuyjykoohewxzeg',
 'vh_make_model_cgighhnwnkxluccz',
 'vh_make_model_cgrdxjyaxssrszjz',
 'vh_make_model_cictcfpmfdmknnye',
 'vh_make_model_ciuxczxwhwbxdkdf',
 'vh_make_model_clmcokjtplrbzvuh',
 'vh_make_model_clsrzyechukbaeat',
 'vh_make_model_cmmuslxsfluvfyof',
 'vh_make_model_cnicorpxweynumqk',
 'vh_make_model_cnlvybtdupkcwczn',
 'vh_make_model_coufviypetbrtevy',
 'vh_make_model_cqdmtwkacajclcml',
 'vh_make_model_cqewccykrcmvawlo',
 'vh_make_model_csxjshhnfbtgjcgm',
 'vh_make_model_ctachoeiozcpkmst',
 'vh_make_model_cufklbvsirnawzmv',
 'vh_make_model_cuxaapvakeemmbaa',
 'vh_make_model_cxxzogxxkmkjwqui',
 'vh_make_model_dbtkrhmbfxpkqbau',
 'vh_make_model_dcjzdpoxqvgnjpmi',
 'vh_make_model_demgvtbzilochupd',
 'vh_make_model_dgwbxitzfzbegnoc',
 'vh_make_model_dgwtezteqyzzylho',
 'vh_make_model_dhjmmmtnpcnalzna',
 'vh_make_model_dhxftxnxtxlgqcqb',
 'vh_make_model_djxdgbpuyerxgrmx',
 'vh_make_model_djyptluftbfkxtjd',
 'vh_make_model_dkgrgmlhhtnvzmps',
 'vh_make_model_dlbnpwopifytzerl',
 'vh_make_model_dllcylnkzeegtsgr',
 'vh_make_model_dlrodwgixwmoquny',
 'vh_make_model_dlrvgwmumnwcjixm',
 'vh_make_model_dnvcqpxxzahdhbvy',
 'vh_make_model_dohofttmidfqjozb',
 'vh_make_model_dohuwjuguzyvqaqg',
 'vh_make_model_doohwubeqhbkevhr',
 'vh_make_model_dpcnodgqfivkhxvn',
 'vh_make_model_dpklliwcxycpfriu',
 'vh_make_model_dqqtizjjhjmqdqqb',
 'vh_make_model_drptidaltxzxopwv',
 'vh_make_model_drpwkafcvcypyrmw',
 'vh_make_model_dsqmtbudvjtnnjwq',
 'vh_make_model_dtdrfrtruyhvbztx',
 'vh_make_model_dtpbahjtnmyuxqno',
 'vh_make_model_dvshwarqhxfcgwfd',
 'vh_make_model_dweqmfoluivgiayj',
 'vh_make_model_dwhlbcevejvegsob',
 'vh_make_model_dwsasdexwmpsmowl',
 'vh_make_model_dyzvyrmcdyybbddd',
 'vh_make_model_dzbwjjmruyqxyvms',
 'vh_make_model_ebdcmhmtqnfkaalo',
 'vh_make_model_edlxghhjgpmvhabz',
 'vh_make_model_efiskxgaocgqqjvr',
 'vh_make_model_ehapkksqqcbofeid',
 'vh_make_model_ehtbxdjhvcwdapsg',
 'vh_make_model_eivjhovgfnfctgjy',
 'vh_make_model_ejbxcyhffvcouoxd',
 'vh_make_model_ejeggxbwhufjtjhd',
 'vh_make_model_ejlwzigdhipvpndt',
 'vh_make_model_eokuiduvnrtzavmr',
 'vh_make_model_epbwnmcyogpybxlm',
 'vh_make_model_esilvarzflhfmjhh',
 'vh_make_model_eslneidrjqwzpqhd',
 'vh_make_model_ettwalwfkzvwdasa',
 'vh_make_model_eudwptcohxaazhpt',
 'vh_make_model_ewkcexkqpsyfnugi',
 'vh_make_model_exkqtrkthhgvjqdl',
 'vh_make_model_eyrwkwxecpzxzscp',
 'vh_make_model_ezaffjpqpacrufvd',
 'vh_make_model_fadjogsnmecatcfb',
 'vh_make_model_fdbwfjqkichwdebq',
 'vh_make_model_fklewvbxuecmupxn',
 'vh_make_model_fkltkgzmjnzqzlqv',
 'vh_make_model_flpmjcetsinyjimc',
 'vh_make_model_fnfpmchfyyqmdtfm',
 'vh_make_model_fnqgfjfkzhfbiicl',
 'vh_make_model_fozvmjndontqoxpg',
 'vh_make_model_fszxbpjtsihsmnqv',
 'vh_make_model_fuddhlszptfmosir',
 'vh_make_model_fuwhdjmdexrstmmo',
 'vh_make_model_fvflhdedljqrcqle',
 'vh_make_model_gapclpflkdsbeorm',
 'vh_make_model_gdaxhrlhuilhiijt',
 'vh_make_model_gdtzpvajphaxanpi',
 'vh_make_model_gdultxlilvdnuwso',
 'vh_make_model_gdzfmtghobzpihgc',
 'vh_make_model_gfhjqtkgvomiygvx',
 'vh_make_model_ggaokfjtqxyctvok',
 'vh_make_model_gguphuccgeqyojbl',
 'vh_make_model_ggzcspiycgszcunf',
 'vh_make_model_giyhzprslgbwsaeu',
 'vh_make_model_gjbalugsikirqoam',
 'vh_make_model_gjblfwqtnckjletn',
 'vh_make_model_gjchrdhbeixppooh',
 'vh_make_model_gjpgirzuabhfpkjd',
 'vh_make_model_gjxmrfgnorpfspbb',
 'vh_make_model_gmzbnaysqjpkzqbt',
 'vh_make_model_gqfadgvnztixxbmv',
 'vh_make_model_gsbeyysssgzgkkuo',
 'vh_make_model_gsooyxmnwsucrksh',
 'vh_make_model_gtvhxebtkefavzhg',
 'vh_make_model_guiimarisyyjqnfg',
 'vh_make_model_gujwvdfcmmqcwxfi',
 'vh_make_model_gwptulznqgygeegy',
 'vh_make_model_gzebcnjcmqioqcjb',
 'vh_make_model_hayciibjzwapccnb',
 'vh_make_model_hcfpedolsygjsofb',
 'vh_make_model_hcoxxbfccserxklx',
 'vh_make_model_hhrmdevbfqiebnum',
 'vh_make_model_hkazsxqvbtmawovu',
 'vh_make_model_hpohizpkyzvwunni',
 'vh_make_model_hselphnqlvecmmyx',
 'vh_make_model_htedybhazfjiueyj',
 'vh_make_model_htppstzpipwjtuia',
 'vh_make_model_huoicgalccftwyvz',
 'vh_make_model_hvjwbevmcmjpnknw',
 'vh_make_model_hwldevoubgzgbhgs',
 'vh_make_model_hywzsmogbhnfcaxk',
 'vh_make_model_iditakunbaxfjcmc',
 'vh_make_model_ieqgavmmxulqlvvl',
 'vh_make_model_ifrzhyqsimoeljaa',
 'vh_make_model_iljhlfeengkciosq',
 'vh_make_model_innngarflvbnwntw',
 'vh_make_model_iqepotyqjqeebzix',
 'vh_make_model_ismjlsoibleinjdp',
 'vh_make_model_iulvirmzdntweaee',
 'vh_make_model_iwhqpdfuhrsxyqxe',
 'vh_make_model_ixbrfaoerogqomah',
 'vh_make_model_ixwsqebjjdlxcqsq',
 'vh_make_model_jakvzvdollijyhwm',
 'vh_make_model_jcefoutonncubdss',
 'vh_make_model_jeckddxjsdolnuhe',
 'vh_make_model_jedhlhdmkdprvyex',
 'vh_make_model_jepialiqqsttgcid',
 'vh_make_model_jgkpiuuctpywtrlh',
 'vh_make_model_jhdjdpthkztnjvmb',
 'vh_make_model_jixkbeuswaznqplh',
 'vh_make_model_jiyzqszfywhdfsil',
 'vh_make_model_jjjvjaxpzvlbryfd',
 'vh_make_model_jjycmklnkdivnypu',
 'vh_make_model_jkarjtlhihuxqzfm',
 'vh_make_model_jkguypwgxebmtnkx',
 'vh_make_model_jlxizhsfukrheysf',
 'vh_make_model_jmycebfjwrkqwsxi',
 'vh_make_model_johsjccpkithubii',
 'vh_make_model_joosvbazdbslkqgx',
 'vh_make_model_jrwdpzrmxqlzzepk',
 'vh_make_model_jrwemlawxsvnwrxv',
 'vh_make_model_jsudrcgsrfddwixw',
 'vh_make_model_kalfshwbcuoobdwe',
 'vh_make_model_kbgblyclstrmicux',
 'vh_make_model_kcjttmlajpvbntkn',
 'vh_make_model_kdsbtuikoaulynsu',
 'vh_make_model_kfurwythfncqbrxs',
 'vh_make_model_kgezpfvpmpmdicts',
 'vh_make_model_kguahfjnmerrbtpp',
 'vh_make_model_kjdumkaiaeblbxtt',
 'vh_make_model_kjhuznifzeghfdra',
 'vh_make_model_kkxluqnhrmwkfqnh',
 'vh_make_model_kmlnlefquqpparsa',
 'vh_make_model_kowgdytyvjhvcmta',
 'vh_make_model_kpciudedjlrqsfte',
 'vh_make_model_kqxycgbergacgcei',
 'vh_make_model_ktrfapbareyzyyyq',
 'vh_make_model_kvcddisqpkysmvvo',
 'vh_make_model_kwxjejihbgmtnagf',
 'vh_make_model_kzqcxkrdytalrphb',
 'vh_make_model_kzwthrslljkmbqur',
 'vh_make_model_lcokgbxbqigkqzcw',
 'vh_make_model_ldejndeewhhlcvgc',
 'vh_make_model_ldkzuxzespcgajev',
 'vh_make_model_lfzbrhthlxhnmhva',
 'vh_make_model_lhyhsxrxdftbsavk',
 'vh_make_model_llkwlxfjdmrqmdgq',
 'vh_make_model_lmqoiaqyftqublmk',
 'vh_make_model_lqkdgbosdzrtitgx',
 'vh_make_model_lqqciehjjdtelpwa',
 'vh_make_model_lwrjcljtxkokvnes',
 'vh_make_model_lxvjgyjdszxtcryf',
 'vh_make_model_mbjevmuapzxqjnwg',
 'vh_make_model_mbytpqiuixyvpaab',
 'vh_make_model_mcadxmmocjhzzbtt',
 'vh_make_model_mcloznejvtelpcan',
 'vh_make_model_mcuawemlwwgaiesn',
 'vh_make_model_mdxtphkujabwpjeu',
 'vh_make_model_meratbpknllwoefn',
 'vh_make_model_mizxbkgdiuoehddq',
 'vh_make_model_mjpgppxzelxrbcnt',
 'vh_make_model_mkbpzddzmalsleud',
 'vh_make_model_mpmchhrcazhsvjgc',
 'vh_make_model_mshhupropfijhilz',
 'vh_make_model_mymdahqxtsywqpdn',
 'vh_make_model_nbxjozrynlospbso',
 'vh_make_model_ngksfbgkdeufmhfy',
 'vh_make_model_ngombkqqomblyxwv',
 'vh_make_model_nhoebceeiacnmvym',
 'vh_make_model_nhwgapjtnadqqaul',
 'vh_make_model_nilvygybpajtnxnr',
 'vh_make_model_nkueyjctyasmotny',
 'vh_make_model_nmhahirmbvqxhxgg',
 'vh_make_model_nnzwevftfeodipkn',
 'vh_make_model_nofmcfnaiuzlqgrk',
 'vh_make_model_nolayrxwnjwzgtoo',
 'vh_make_model_nrmzpcqkbzgmsdeo',
 'vh_make_model_nruhduwvuytxnfvh',
 'vh_make_model_nsgbpbjvswwlhvmm',
 'vh_make_model_nsymgnybdjqxudvj',
 'vh_make_model_nwaavqeweeqaryzv',
 'vh_make_model_nxwedpnhirijkodc',
 'vh_make_model_nyrtstlobluggnkw',
 'vh_make_model_nzanewsbtbnpgrom',
 'vh_make_model_nzxlhibmhrtafeav',
 'vh_make_model_obvxygchobqafuzw',
 'vh_make_model_odpuaztxnyumdvvc',
 'vh_make_model_ofrkezlcbbluncri',
 'vh_make_model_ohgtowaarzphsifb',
 'vh_make_model_ohxrgpugowiyinhv',
 'vh_make_model_oihtzffwsrwsjnfu',
 'vh_make_model_ojribuhtopqgkqpp',
 'vh_make_model_olupoctwepebdqqo',
 'vh_make_model_onzjhhtppsfaiacz',
 'vh_make_model_optzzqvbwwriedfo',
 'vh_make_model_oryfrzxilushvigq',
 'vh_make_model_otrziwxmbpndmyaa',
 'vh_make_model_owkiszjuntmwilff',
 'vh_make_model_owxrlgxbigikfgtm',
 'vh_make_model_ozmdlzfsareqmkon',
 'vh_make_model_ozpyjjijxdpztngv',
 'vh_make_model_pdljbgzzhxrhnqmu',
 'vh_make_model_pfwcfdvpkuyucnkn',
 'vh_make_model_pheduvdlnmrchihf',
 'vh_make_model_phprbhssfhrtbeue',
 'vh_make_model_pjbnwqhnqczouirt',
 'vh_make_model_ponwkmeaxagundzq',
 'vh_make_model_pqpqthiapbycbhor',
 'vh_make_model_prtnwsypyfnshpqx',
 'vh_make_model_pselomoxubpkknqo',
 'vh_make_model_ptaxsjwbissrpvdm',
 'vh_make_model_ptbudvgjgycmmsdq',
 'vh_make_model_pustczakchcimwuy',
 'vh_make_model_pvrjjyumueakzstw',
 'vh_make_model_pyholyswkkqjmxlj',
 'vh_make_model_pyykjiriqrhjduly',
 'vh_make_model_qbkipjmisqllqwzy',
 'vh_make_model_qbztetcodwhfmoyg',
 'vh_make_model_qcykqtxlqnbcqfct',
 'vh_make_model_qdvjpkftaveygusd',
 'vh_make_model_qewzxgvvhqhkfcxe',
 'vh_make_model_qfvolfbvalczrcko',
 'vh_make_model_qgnqfinpenszbzig',
 'vh_make_model_qidpxyunryowizua',
 'vh_make_model_qjkwsppqbsgsvjwa',
 'vh_make_model_qnesuhpxsptzihzg',
 'vh_make_model_qnixeczkijjyiprb',
 'vh_make_model_qpcebxmotqhildhx',
 'vh_make_model_qqmkwgdqaimwcbxo',
 'vh_make_model_quslbttvcitxzeiy',
 'vh_make_model_qvwenzdmnwecdiql',
 'vh_make_model_qwcrrrebwyeauczj',
 'vh_make_model_qxtqrwxfvuenelml',
 'vh_make_model_qzgaezfhutbcnkuf',
 'vh_make_model_rabwrzdzwjjdhbmx',
 'vh_make_model_rbxibrjokiihgfjb',
 'vh_make_model_rcxmbwwsxkkkyyjs',
 'vh_make_model_reolzfmikorzxstf',
 'vh_make_model_rgrpzewhrznrqrna',
 'vh_make_model_rguedwefqmzdxowu',
 'vh_make_model_rhxboadaoyvvgflk',
 'vh_make_model_rjhfsrwtoqfqvuqu',
 'vh_make_model_rlkrrmxxdgaxangi',
 'vh_make_model_rnrkbyojyiepdvqv',
 'vh_make_model_rqaprgqcktgrlxnv',
 'vh_make_model_rqklbykswxeuovdn',
 'vh_make_model_rrlvhbnzrdtphqnl',
 'vh_make_model_rrqbtdjvuwwxtusj',
 'vh_make_model_rthsjeyjgdlmkygk',
 'vh_make_model_rulqevsymrlwrsrz',
 'vh_make_model_ruposftqgswlcyou',
 'vh_make_model_rwrevaiebpmviwqz',
 'vh_make_model_rwtwnvhjqabvovnz',
 'vh_make_model_rxyndewyvbophaku',
 'vh_make_model_rzjssfxzzoddvgdc',
 'vh_make_model_saempmkfulqhwfqk',
 'vh_make_model_sboaeuuuhpsjujpz',
 'vh_make_model_sdottmimvqvfhzlk',
 'vh_make_model_sgknghheolfpzuid',
 'vh_make_model_sguprofjftozaujc',
 'vh_make_model_sioekxjbocpzrjzi',
 'vh_make_model_smynsodmtrrubpqq',
 'vh_make_model_snpaaoiipfuxmvol',
 'vh_make_model_snsnxmucuccvqfvz',
 'vh_make_model_spbjeokdemicpdey',
 'vh_make_model_srgqbkjrwdbikmzq',
 'vh_make_model_sutdaojcvfqmjnwg',
 'vh_make_model_svcvmlpsqtzbrmnz',
 'vh_make_model_svmjzfcsvgxiwwjt',
 'vh_make_model_swjkmyqytzxjwgag',
 'vh_make_model_swxgkelaxkoffszz',
 'vh_make_model_synvsxhrexuyxpre',
 'vh_make_model_szduoosmrfqduakm',
 'vh_make_model_szlkmablxrjoubla',
 'vh_make_model_szvfwsizhxrbklhz',
 'vh_make_model_taovawittfogygzi',
 'vh_make_model_tceovgpqjjopitor',
 'vh_make_model_tddtoayhfpdtxokp',
 'vh_make_model_tdgkjlphosocwbgu',
 'vh_make_model_tdldeeccsirqwpcj',
 'vh_make_model_tdozuksvtvtqcykp',
 'vh_make_model_tduddcyerrjazjsh',
 'vh_make_model_tegzsblugaczvdmy',
 'vh_make_model_tgfhgapnsxiewemd',
 'vh_make_model_tkqxtjbbrzagooya',
 'vh_make_model_tksyxmdgogmokuxv',
 'vh_make_model_tlhipnhcbdhvhgyw',
 'vh_make_model_toqaaqswchaiyhsk',
 'vh_make_model_tsfyxgkwdidzgzpg',
 'vh_make_model_tupmlwnkgjcgcmuv',
 'vh_make_model_twhtulgwsricneea',
 'vh_make_model_tyktnjdtbrucursh',
 'vh_make_model_uahuglbjdtacoqjt',
 'vh_make_model_uamadghmregezetz',
 'vh_make_model_ubttjiaeeuwzcclq',
 'vh_make_model_uclvvrkbezlvaulu',
 'vh_make_model_udlfdefgndowttah',
 'vh_make_model_ueujqvpwszzhovbj',
 'vh_make_model_ufpwnqycocwwbgqi',
 'vh_make_model_ufqiiuoxasjwxqbf',
 'vh_make_model_ugwrafmvdavbsrzl',
 'vh_make_model_uhkhghxuorryhlis',
 'vh_make_model_uhqwwluaswbuqqjc',
 'vh_make_model_ujgldnoigollndkj',
 'vh_make_model_ukqbsscpgbfatdhs',
 'vh_make_model_unlqqlfvajjczyks',
 'vh_make_model_unmxysjyilftwsvy',
 'vh_make_model_uotpflqyvprslxjc',
 'vh_make_model_urullqqbaabxllxl',
 'vh_make_model_usjukvawgoqplrph',
 'vh_make_model_usmpkujeknoxdqrc',
 'vh_make_model_uureltetaotxxdji',
 'vh_make_model_uwwrbkmjbjyxutfq',
 'vh_make_model_uxfxpraspeoqtmbg',
 'vh_make_model_uzfgpmnazksmudrw',
 'vh_make_model_uzmldekmvczimsrj',
 'vh_make_model_uzqtsirvtxcfqnbp',
 'vh_make_model_uzrhbfduaqijosql',
 'vh_make_model_vbnagxwgmwirhnjt',
 'vh_make_model_vdetuihriafhetdl',
 'vh_make_model_vdwfxzfzbybhsmay',
 'vh_make_model_vikmjrynreazqubj',
 'vh_make_model_vjarzxzevsdnftcl',
 'vh_make_model_vjxfnjqgvugwjhia',
 'vh_make_model_vnxfpyxuciadydrl',
 'vh_make_model_vokpwtikxckeemdi',
 'vh_make_model_vpvsuuudxglarezp',
 'vh_make_model_vpvtqlxqaiejzrqo',
 'vh_make_model_vudjowytbogxkrcy',
 'vh_make_model_vxqfscklywhurrjp',
 'vh_make_model_vxumxjoeywcphfoo',
 'vh_make_model_vxvmxuncsxygbrzd',
 'vh_make_model_wchxrbrhstsmhdsk',
 'vh_make_model_wcyahxrmwqvhmadq',
 'vh_make_model_wehkqzwvbeonajcu',
 'vh_make_model_wgruytvmfzalzrtb',
 'vh_make_model_whcqrtwarljaqocm',
 'vh_make_model_wiyyghhcxezudyxg',
 'vh_make_model_wotupxkyxwcienzd',
 'vh_make_model_wpsyqubfrhdspxkx',
 'vh_make_model_wrzuftzqwoiwsmfc',
 'vh_make_model_wsfgxnwhxftjhpxw',
 'vh_make_model_wtxtgodhmneofvzz',
 'vh_make_model_wxzfbqtarfurwcfw',
 'vh_make_model_wyopladghryqlrlb',
 'vh_make_model_wyoxchtoecahbyjm',
 'vh_make_model_wyqgeeclrqbihfpk',
 'vh_make_model_xaklvfxsplowrglp',
 'vh_make_model_xcvdausiwfrjukgn',
 'vh_make_model_xdeuhuabvdhjipnp',
 'vh_make_model_xebooruxiuwbpzdc',
 'vh_make_model_xewlloxrajhpbuwy',
 'vh_make_model_xgikerzyofvqsmnt',
 'vh_make_model_xgvsuftfggoojbdp',
 'vh_make_model_xhczitnzxmxxebeq',
 'vh_make_model_xiesnbkcyzrpzlyq',
 'vh_make_model_xiwnirovwicymtif',
 'vh_make_model_xjaddkudsebowzen',
 'vh_make_model_xjnjmxyqemqqiejp',
 'vh_make_model_xjsozzwcppavldee',
 'vh_make_model_xkqlmdeookbxzzhv',
 'vh_make_model_xkzehzohmfrsmolg',
 'vh_make_model_xoetbcowyxwukawr',
 'vh_make_model_xpxsjmglcvcsxwdy',
 'vh_make_model_xqydufxhniyjunrl',
 'vh_make_model_xsmqbeukcqahbfgl',
 'vh_make_model_xsonelzsqbpcodxe',
 'vh_make_model_xuioboiuzasnmuva',
 'vh_make_model_xxppvuhwmnezefxy',
 'vh_make_model_xyuwuxlpirkzkqdb',
 'vh_make_model_xzdsapxqliboezbc',
 'vh_make_model_ybbdbpunwekygnto',
 'vh_make_model_ybmkbrazyartpatx',
 'vh_make_model_ybsenzutfrjternf',
 'vh_make_model_yfalryaixpzfoihd',
 'vh_make_model_yglorajvvrsviget',
 'vh_make_model_ygqjgwzgeierkcpj',
 'vh_make_model_ygsnfanduarpqvrn',
 'vh_make_model_yhcelpjnbxpsmoez',
 'vh_make_model_ylboorftnzombypn',
 'vh_make_model_yljpdxzmshdpmyhl',
 'vh_make_model_yqrgxthbbzmruvwy',
 'vh_make_model_ysonsqntnqnqagnn',
 'vh_make_model_yttvzqeuddvehiqu',
 'vh_make_model_yvlkrzgjhwrlyihc',
 'vh_make_model_ywnimisaozuyjomi',
 'vh_make_model_yxzgvihpyqafgdmy',
 'vh_make_model_yybjogamsfqljfpu',
 'vh_make_model_yzdbjmwwtofxmpaz',
 'vh_make_model_yzxgnwgpnrdprtbh',
 'vh_make_model_zagpnfpbwgeyeufr',
 'vh_make_model_zakviitdfvxsgkow',
 'vh_make_model_zbprczwzmlxgqykc',
 'vh_make_model_zcqptmhakcmihiry',
 'vh_make_model_zedpyrfkmhzqxmaz',
 'vh_make_model_zfylmujpvzgqqfxo',
 'vh_make_model_zkreetxvsoihwkgo',
 'vh_make_model_zojbyremtnxajomo',
 'vh_make_model_zoypfizhpbtpjwpv',
 'vh_make_model_zqruwnlzuefcpqjm',
 'vh_make_model_zqswdfwtkyehitft',
 'vh_make_model_zrfduayrhhofpqtt',
 'vh_make_model_zspzyfdefowgwddf',
 'vh_make_model_ztwsplndgicacmuu',
 'vh_make_model_zwutaiivgrxnrwat',
 'vh_make_model_zxvcbwcwoqnkxxbs',
 'vh_make_model_zydfvjqmmwhyfuyy',
 'vh_make_model_zzlzzujtugbfpsvv',
 'pol_usage_AllTrips',
 'pol_usage_Professional',
 'pol_usage_Retired',
 'pol_usage_WorkPrivate',
 'drv_sex2_0',
 'drv_sex2_F',
 'drv_sex2_M',
 'vh_fuel_Diesel',
 'vh_fuel_Gasoline',
 'vh_fuel_Hybrid',
 'year',
 'pol_duration',
 'pol_pay_freq',
 'pol_payd',
 'drv_sex1',
 'drv_age1',
 'drv_age_lic1',
 'drv_drv2',
 'vh_age',
 'vh_type',
 'population',
 'pol_no_claims_discount',
 'drv_age2',
 'drv_age_lic2',
 'vh_speed',
 'vh_value',
 'vh_weight',
 'town_surface_area']

In [96]:
def preprocess_x(dataframe, l=list_vh_make_model, c=all_cols):
    claims = dataframe.copy(deep=True)
    claims['pol_pay_freq'] = claims['pol_pay_freq'].replace( {'Biannual': 2, 'Yearly': 1, 'Monthly': 12, 'Quarterly': 4} )
    claims['pol_payd'] = claims['pol_payd'].replace( {'No': 0, 'Yes': 1} )
    claims['drv_sex1'] = claims['drv_sex1'].replace( {'M': 1, 'F': 0} )
    claims['vh_type'] = claims['vh_type'].replace( {'Tourism': 1, 'Commercial': 0} )
    claims['drv_drv2'] = claims['drv_drv2'].replace( {'No': 0, 'Yes': 1} )
    
    objects = claims.select_dtypes(['object'])
    categorical = claims.select_dtypes(['int64'])
    continuous = claims.select_dtypes(['float64'])

    objects_filled = SimpleImputer(missing_values=np.nan, strategy='most_frequent').fit_transform(objects)
    categorial_filled = SimpleImputer(missing_values=np.nan, strategy='median').fit_transform(categorical)
    continuous_filled = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(continuous)

    objects_filled.drop(columns=['vh_make_model'])

    objects_filled = pd.DataFrame(objects_filled, columns=objects.columns.to_list())
    categorial_filled = pd.DataFrame(categorial_filled, columns=categorical.columns.to_list())
    continuous_filled = pd.DataFrame(continuous_filled, columns=continuous.columns.to_list())

    design_matrix = pd.get_dummies(objects_filled, columns=['pol_usage', 'drv_sex2', 'vh_fuel'], dtype=int) #'vh_make_model', 
    objects_design = design_matrix.drop(columns=['id_policy'])

    # for item in l:
    #     if not item in objects_design:
    #         objects_design[item] = 0

    df = pd.concat([objects_design, categorial_filled, continuous_filled], axis = 1)

    # df = df[c]

    return df

In [97]:
x_train, x_test, y_train, y_test = preprocess_xy(claims)

C:\Users\kyle\AppData\Local\Temp\ipykernel_15320\3866130775.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  claims['pol_pay_freq'] = claims['pol_pay_freq'].replace( {'Biannual': 2, 'Yearly': 1, 'Monthly': 12, 'Quarterly': 4} )
C:\Users\kyle\AppData\Local\Temp\ipykernel_15320\3866130775.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  claims['pol_payd'] = claims['pol_payd'].replace( {'No': 0, 'Yes': 1} )
C:\Users\kyle\AppData\Local\Temp\ipykernel_15320\3866130775.py:5: FutureWarning: Downcasting behavior in `replace` is depr

In [98]:
er = VotingRegressor([('kn', KNeighborsRegressor(n_neighbors=30)), ('sv', svm.SVR())])
er.fit(x_train, y_train)
y_pred = er.predict(x_test)
print(RMSE(y_pred, y_test))

ValueError: could not convert string to float: 'guiimarisyyjqnfg'

In [ ]:
x_train.shape

(11338, 497)

In [ ]:
def evaluate(claims, model):
    x_train, x_test, y_train, y_test = preprocess_xy(claims)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    return np.sqrt(metrics.mean_squared_error(y_test, y_pred))

In [ ]:
print(evaluate(claims, er))

C:\Users\kyle\AppData\Local\Temp\ipykernel_15320\759866183.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  claims['pol_pay_freq'] = claims['pol_pay_freq'].replace( {'Biannual': 2, 'Yearly': 1, 'Monthly': 12, 'Quarterly': 4} )
C:\Users\kyle\AppData\Local\Temp\ipykernel_15320\759866183.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  claims['pol_payd'] = claims['pol_payd'].replace( {'No': 0, 'Yes': 1} )
C:\Users\kyle\AppData\Local\Temp\ipykernel_15320\759866183.py:5: FutureWarning: Downcasting behavior in `replace` is depreca

1372.762275666161


In [ ]:
def predict(x, model):
    x_new = preprocess_x(x)
    return model.predict(x_new)

In [ ]:
data = pd.read_csv("./Qualification_Package/Submission_Data.csv")

In [ ]:
print(data.shape)

data_processed = preprocess_x(data)

print(data_processed.shape)

(4140, 24)
(4140, 382)


C:\Users\kyle\AppData\Local\Temp\ipykernel_15320\3229044543.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  claims['pol_pay_freq'] = claims['pol_pay_freq'].replace( {'Biannual': 2, 'Yearly': 1, 'Monthly': 12, 'Quarterly': 4} )
C:\Users\kyle\AppData\Local\Temp\ipykernel_15320\3229044543.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  claims['pol_payd'] = claims['pol_payd'].replace( {'No': 0, 'Yes': 1} )
C:\Users\kyle\AppData\Local\Temp\ipykernel_15320\3229044543.py:5: FutureWarning: Downcasting behavior in `replace` is depr

In [ ]:
trained = set(x_train.columns.to_list())

In [ ]:
real = set(data_processed.columns.to_list())

In [ ]:
print(trained - real)
print(real - trained)

{'vh_make_model_ylboorftnzombypn', 'vh_make_model_xqydufxhniyjunrl', 'vh_make_model_mkbpzddzmalsleud', 'vh_make_model_ejlwzigdhipvpndt', 'vh_make_model_fnfpmchfyyqmdtfm', 'vh_make_model_hvjwbevmcmjpnknw', 'vh_make_model_pvrjjyumueakzstw', 'vh_make_model_qwcrrrebwyeauczj', 'vh_make_model_xsonelzsqbpcodxe', 'vh_make_model_vxvmxuncsxygbrzd', 'vh_make_model_tgfhgapnsxiewemd', 'vh_make_model_dqqtizjjhjmqdqqb', 'vh_make_model_cgighhnwnkxluccz', 'vh_make_model_tsfyxgkwdidzgzpg', 'vh_make_model_lhyhsxrxdftbsavk', 'vh_make_model_wrzuftzqwoiwsmfc', 'vh_make_model_ybbdbpunwekygnto', 'vh_make_model_tksyxmdgogmokuxv', 'vh_make_model_fkltkgzmjnzqzlqv', 'vh_make_model_xiwnirovwicymtif', 'vh_make_model_ywnimisaozuyjomi', 'vh_make_model_fvflhdedljqrcqle', 'vh_make_model_taovawittfogygzi', 'vh_make_model_cictcfpmfdmknnye', 'vh_make_model_ayeiibefzqqbyksg', 'vh_make_model_zojbyremtnxajomo', 'vh_make_model_bikffjqejohkyhat', 'vh_make_model_vxqfscklywhurrjp', 'vh_make_model_ngksfbgkdeufmhfy', 'vh_make_mode

In [ ]:

x_new = preprocess_x(data)

predictions = er.predict(x_new)